# RAG
Retrieval Augmented Generation, 검색 증강 생성

LLM에게 외부 데이터를 context로 활용할 수 있도록 처리 하는 과정

## 사전처리 단계

1. 문서 불러오기(Loading)
2. 텍스트 나누기(Splitting)
3. 숫자로 바꾸기(Embedding)
4. 저장하기(VectorStore)

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Goldfish are popular pets for beginners, requiring relatively simple care.",
        metadata={"source": "fish-pets-doc"},
    ),
    Document(
        page_content="Parrots are intelligent birds capable of mimicking human speech.",
        metadata={"source": "bird-pets-doc"},
    ),
    Document(
        page_content="Rabbits are social animals that need plenty of space to hop around.",
        metadata={"source": "mammal-pets-doc"},
    ),
]  # 1. 문서 불러오기 - 2. 텍스트 나누기까지 완료된 상태

In [3]:
from langchain_openai import OpenAIEmbeddings

# OpenAI의 임베딩 모델
embedding = OpenAIEmbeddings()

## 검색 증강 단계
